In [3]:
import pandas as pd

# Path to your label.lst file
lst_file_path = 'ExpW\data\label\label.lst'

# Define column names based on the structure of your .lst file
column_names = ['image_name', 'label', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'extra_label']

# Load the .lst file into a Pandas DataFrame
labels = pd.read_csv(lst_file_path, sep=' ', names=column_names)

# Check if the data has been loaded correctly
print(labels.head())


<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\aftab\AppData\Local\Temp\ipykernel_8744\174787535.py:4: SyntaxWarning: invalid escape sequence '\d'
  lst_file_path = 'ExpW\data\label\label.lst'


            image_name  label  xmin  ymin  xmax  ymax  confidence  extra_label
0  angry_actor_104.jpg      0    28   113   226   141     22.9362            0
1  angry_actor_109.jpg      0    31   157   345   219     50.3056            0
2  angry_actor_120.jpg      1    53    53   372   372     13.9434            2
3   angry_actor_13.jpg      0    77    51   362   388     85.8104            3
4  angry_actor_132.jpg      0    95    31   412   476     82.3948            0


In [4]:
# Check for missing values in the 'label' column
print(labels['label'].isna().sum())

# If any rows contain NaN, show those rows
print(labels[labels['label'].isna()])


0
Empty DataFrame
Columns: [image_name, label, xmin, ymin, xmax, ymax, confidence, extra_label]
Index: []


In [5]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets, using stratification on labels
train_df, val_df = train_test_split(labels, test_size=0.2, stratify=labels['label'], random_state=42)

# Check the distribution of labels in the train and validation sets
print(train_df['label'].value_counts())
print(val_df['label'].value_counts())


label
0    42773
1    15036
2     6134
3     3514
4     2110
5     1349
6      954
7      658
8      522
9      384
Name: count, dtype: int64
label
0    10693
1     3759
2     1534
3      879
4      528
5      337
6      238
7      165
8      130
9       96
Name: count, dtype: int64


In [6]:
# Convert the labels to strings
train_df['label'] = train_df['label'].astype(str)
val_df['label'] = val_df['label'].astype(str)


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to the image folder (use raw string literals)
image_folder = 'ExpW\data\image\origin'

# Initialize ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Flow images from the directory for training and validation
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=image_folder,
    x_col='image_name',
    y_col='label',
    target_size=(224, 224),  # Adjust as per your model's requirements
    batch_size=32,
    class_mode='categorical',  # or 'binary' based on your labels
    subset='training'
)

val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=image_folder,
    x_col='image_name',
    y_col='label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # or 'binary' based on your labels
    subset='validation'
)


<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\aftab\AppData\Local\Temp\ipykernel_8744\2089009227.py:4: SyntaxWarning: invalid escape sequence '\d'
  image_folder = 'ExpW\data\image\origin'


Found 58748 validated image filenames belonging to 10 classes.
Found 3671 validated image filenames belonging to 10 classes.


In [15]:
# Defining CNN Model
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# Load the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of ResNet50
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)  # Assuming 7 facial expression classes

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 63s 1us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 74,973,578 (286.00 MB)

 Trainable params: 51,385,866 (196.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Get the number of steps for training and validation
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = val_generator.n // val_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=val_generator,
    validation_steps=validation_steps
)


Epoch 1/20


c:\Users\aftab\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1232/1835 ━━━━━━━━━━━━━━━━━━━━ 1:57:43 12s/step - accuracy: 0.5575 - loss: 1.4525